# Load Pickles Files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pickle

### Merge crop and trades DataFrame

In [3]:
unpickled_df = pd.read_pickle("./big_ass_df.pickle")

In [4]:
unpickled_df

,area_code,area_crops,item_code_crops,item_crops,element_code_crops,element_crops,year,unit_crops,value_crops,flag,item_key,area_trades,item_code_trades,item_trades,element_code_trades,element_trades,unit_trades,value_trades
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,ha,0.0,NaN,almond,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,0.0
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,ha,0.0,NaN,almond,Afghanistan,231,Almonds shelled,5922,Export Value,1000 US$,0.0
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,1975,tonnes,0.0,NaN,almond,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,0.0
3,2,Afghanistan,221,"Almonds, with shell",5510,Production,1975,tonnes,0.0,NaN,almond,Afghanistan,231,Almonds shelled,5922,Export Value,1000 US$,0.0
4,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,ha,5900.0,NaN,almond,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,642.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477830,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,5510,Production,2016,tonnes,81965547.0,NaN,veget,Net Food Importing Developing Countries,474,"Vegetables, temporarily preserved",5922,Export Value,1000 US$,71357.0
11477831,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,5510,Production,2016,tonnes,81965547.0,NaN,veget,Net Food Importing Developing Countries,1296,Waxes vegetable,5610,Import Quantity,tonnes,603.0
11477832,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,5510,Production,2016,tonnes,81965547.0,NaN,veget,Net Food Importing Developing Countries,1296,Waxes vegetable,5622,Import Value,1000 US$,3024.0
11477833,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,5510,Production,2016,tonnes,81965547.0,NaN,veget,Net Food Importing Developing Countries,1296,Waxes vegetable,5910,Export Quantity,tonnes,11.0


In [5]:
# clean unpickled_df
unpickled_df = unpickled_df.drop(['element_code_crops','flag','area_trades','element_code_trades'],axis=1)

In [15]:
unpickled_df

,area_code,area_crops,item_code_crops,item_crops,element_crops,year,unit_crops,value_crops,item_key,item_code_trades,item_trades,element_trades,unit_trades,value_trades
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0
2,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0
3,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0,almond,231,Almonds shelled,Export Quantity,tonnes,642.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477830,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,474,"Vegetables, temporarily preserved",Export Value,1000 US$,71357.0
11477831,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Quantity,tonnes,603.0
11477832,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Value,1000 US$,3024.0
11477833,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Export Quantity,tonnes,11.0


In [20]:
unpickled_df.item_crops.nunique()

129

In [21]:
df_crops_dummy = pd.get_dummies(unpickled_df.element_crops)

In [22]:
df_crops_dummy

,Area harvested,Production,Yield
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
11477830,0,1,0
11477831,0,1,0
11477832,0,1,0
11477833,0,1,0


In [9]:
df_trades_dummy = pd.get_dummies(unpickled_df.element_trades)

In [35]:
pd.pivot_table(unpickled_df,values=['value_crops'],index=['area_crops','item_crops','year'],columns=['element_crops']).reset_index()

area_crops           item_crops  year    value_crops  \
element_crops                                         Area harvested   
0              Afghanistan  Almonds, with shell  1975            0.0   
1              Afghanistan  Almonds, with shell  1976         5900.0   
2              Afghanistan  Almonds, with shell  1977         6000.0   
3              Afghanistan  Almonds, with shell  1978         6000.0   
4              Afghanistan  Almonds, with shell  1979         6000.0   
...                    ...                  ...   ...            ...   
563608            Zimbabwe                Wheat  2013        21753.0   
563609            Zimbabwe                Wheat  2014        22131.0   
563610            Zimbabwe                Wheat  2015        22114.0   
563611            Zimbabwe                Wheat  2016        22094.0   
563612            Zimbabwe                Wheat  2017        22070.0   

                                   
element_crops Production    Yield  
0                    0.0      NaN  
1                 9800.0  16610.0  
2                 9000.0  15000.0  
3                12000.0  20000.0  
4                10500.0  17500.0  
...                  ...      ...  
563608           50966.0  23430.0  
563609           48595.0  21958.0  
563610           45302.0  20486.0  
563611           42008.0  19014.0  
563612           38715.0  17542.0  

[563613 rows x 6 columns]

In [36]:
pd.pivot_table(unpickled_df,values=['value_trades'],index=['area_crops','item_crops','year'],columns=['element_trades']).reset_index()

area_crops           item_crops  year    value_trades  \
element_trades                                         Export Quantity   
0               Afghanistan  Almonds, with shell  1975        0.000000   
1               Afghanistan  Almonds, with shell  1976      642.000000   
2               Afghanistan  Almonds, with shell  1977      286.000000   
3               Afghanistan  Almonds, with shell  1978      518.000000   
4               Afghanistan  Almonds, with shell  1979     1100.000000   
...                     ...                  ...   ...             ...   
567845             Zimbabwe                Wheat  2013      871.666667   
567846             Zimbabwe                Wheat  2014     1461.000000   
567847             Zimbabwe                Wheat  2015     1170.333333   
567848             Zimbabwe                Wheat  2016     1334.666667   
567849             Zimbabwe                Wheat  2017    11718.500000   

                                                           
element_trades Export Value Import Quantity  Import Value  
0                  0.000000             NaN           NaN  
1               2574.000000             NaN           NaN  
2               1220.000000             NaN           NaN  
3               2210.000000             NaN           NaN  
4               4630.000000             NaN           NaN  
...                     ...             ...           ...  
567845           172.666667    83371.666667  41960.666667  
567846           198.000000    92807.666667  43216.000000  
567847           287.333333   111351.333333  49287.666667  
567848           187.666667   108922.333333  40025.333333  
567849          2900.500000    73203.000000  27174.000000  

[567850 rows x 7 columns]

In [10]:
df_trades_dummy

,Export Quantity,Export Value,Import Quantity,Import Value
0,1,0,0,0
1,0,1,0,0
2,1,0,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
11477830,0,1,0,0
11477831,0,0,1,0
11477832,0,0,0,1
11477833,1,0,0,0


In [11]:
df_merged_dummies_trades = pd.concat([unpickled_df,df_trades_dummy],axis=1)
df_merged_dummies_trades.columns = map(lambda name : name.lower().replace(' ', '_'), df_merged_dummies_trades.columns) #Column names to lower case

In [12]:
df_merged_dummies_trades.head()

,area_code,area_crops,item_code_crops,item_crops,element_crops,year,unit_crops,value_crops,item_key,item_code_trades,item_trades,element_trades,unit_trades,value_trades,export_quantity,export_value,import_quantity,import_value
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,1,0,0,0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0,1,0,0
2,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,1,0,0,0
3,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0,1,0,0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0,almond,231,Almonds shelled,Export Quantity,tonnes,642.0,1,0,0,0


In [13]:
df_merged_dummies_crops = pd.concat([unpickled_df,df_crops_dummy],axis=1)
df_merged_dummies_crops.columns = map(lambda name : name.lower().replace(' ', '_'), df_merged_dummies_crops.columns) #Column names to lower case

In [14]:
df_merged_dummies_crops.head()

,area_code,area_crops,item_code_crops,item_crops,element_crops,year,unit_crops,value_crops,item_key,item_code_trades,item_trades,element_trades,unit_trades,value_trades,area_harvested,production,yield
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,1,0,0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,1,0,0
2,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,0,1,0
3,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0,1,0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0,almond,231,Almonds shelled,Export Quantity,tonnes,642.0,1,0,0


In [18]:
# This cell assign the actual corresponding value to the dummies 
df_merged_dummies_trades.loc[df_merged_dummies_trades['import_quantity']==1,'import_quantity'] = df_merged_dummies_trades.loc[df_merged_dummies_trades['import_quantity']==1,'value_trades']
df_merged_dummies_trades.loc[df_merged_dummies_trades['import_value']==1,'import_value'] = df_merged_dummies_trades.loc[df_merged_dummies_trades['import_value']==1,'value_trades']
df_merged_dummies_trades.loc[df_merged_dummies_trades['export_quantity']==1,'export_quantity'] = df_merged_dummies_trades.loc[df_merged_dummies_trades['export_quantity']==1,'value_trades']
df_merged_dummies_trades.loc[df_merged_dummies_trades['export_value']==1,'export_value'] = df_merged_dummies_trades.loc[df_merged_dummies_trades['export_value']==1,'value_trades']

In [19]:
df_merged_dummies_trades

,area_code,area_crops,item_code_crops,item_crops,element_crops,year,unit_crops,value_crops,item_key,item_code_trades,item_trades,element_trades,unit_trades,value_trades,export_quantity,export_value,import_quantity,import_value
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,0.0,0.0,0.0,0.0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0.0,0.0,0.0,0.0
2,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,0.0,0.0,0.0,0.0
3,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0.0,0.0,0.0,0.0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0,almond,231,Almonds shelled,Export Quantity,tonnes,642.0,642.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477830,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,474,"Vegetables, temporarily preserved",Export Value,1000 US$,71357.0,0.0,71357.0,0.0,0.0
11477831,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Quantity,tonnes,603.0,0.0,0.0,603.0,0.0
11477832,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Value,1000 US$,3024.0,0.0,0.0,0.0,3024.0
11477833,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Export Quantity,tonnes,11.0,11.0,0.0,0.0,0.0


In [23]:
df_import_export_trades = df_merged_dummies_trades.groupby(['area_crops','item_crops','year']).agg({
    'area_code' : 'first',
    'item_code_crops' : 'first',
    'export_quantity' : 'sum',
    'export_value' : 'sum',
    'import_quantity' : 'sum',
    'import_value'  : 'sum'
    }).reset_index()

In [24]:
df_import_export_trades

,area_crops,item_crops,year,area_code,item_code_crops,export_quantity,export_value,import_quantity,import_value
0,Afghanistan,"Almonds, with shell",1975,2,221,0.0,0.0,0.0,0.0
1,Afghanistan,"Almonds, with shell",1976,2,221,1926.0,7722.0,0.0,0.0
2,Afghanistan,"Almonds, with shell",1977,2,221,858.0,3660.0,0.0,0.0
3,Afghanistan,"Almonds, with shell",1978,2,221,1554.0,6630.0,0.0,0.0
4,Afghanistan,"Almonds, with shell",1979,2,221,3300.0,13890.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
571860,Zimbabwe,Wheat,2013,181,15,7845.0,1554.0,750345.0,377646.0
571861,Zimbabwe,Wheat,2014,181,15,8766.0,1188.0,835269.0,388944.0
571862,Zimbabwe,Wheat,2015,181,15,10533.0,2586.0,1002162.0,443589.0
571863,Zimbabwe,Wheat,2016,181,15,12012.0,1689.0,980301.0,360228.0


In [95]:
df_merged_dummies_crops.loc[df_merged_dummies_crops['area_harvested']==1,'area_harvested'] = df_merged_dummies_crops.loc[df_merged_dummies_crops['area_harvested']==1,'value_crops']
df_merged_dummies_crops.loc[df_merged_dummies_crops['production']==1,'production'] = df_merged_dummies_crops.loc[df_merged_dummies_crops['production']==1,'value_crops']
df_merged_dummies_crops.loc[df_merged_dummies_crops['yield']==1,'yield'] = df_merged_dummies_crops.loc[df_merged_dummies_crops['yield']==1,'value_crops']

In [96]:
df_merged_dummies_crops

,area_code,area_crops,item_code_crops,item_crops,element_crops,year,unit_crops,value_crops,item_key,item_code_trades,item_trades,element_trades,unit_trades,value_trades,area_harvested,production,yield
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,0.0,0.0,0.0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0.0,0.0,0.0
2,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Quantity,tonnes,0.0,0.0,0.0,0.0
3,2,Afghanistan,221,"Almonds, with shell",Production,1975,tonnes,0.0,almond,231,Almonds shelled,Export Value,1000 US$,0.0,0.0,0.0,0.0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0,almond,231,Almonds shelled,Export Quantity,tonnes,642.0,5900.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11477830,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,474,"Vegetables, temporarily preserved",Export Value,1000 US$,71357.0,0.0,81965547.0,0.0
11477831,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Quantity,tonnes,603.0,0.0,81965547.0,0.0
11477832,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Import Value,1000 US$,3024.0,0.0,81965547.0,0.0
11477833,5817,Net Food Importing Developing Countries,1735,Vegetables Primary,Production,2016,tonnes,81965547.0,veget,1296,Waxes vegetable,Export Quantity,tonnes,11.0,0.0,81965547.0,0.0


In [131]:
df_import_export_crops = df_merged_dummies_crops.groupby(['area_crops','item_crops','year']).agg({
    'area_code' : 'first',
    'item_code_crops' : 'first',
    'area_harvested' : 'mean',
    'production' : 'mean',
    'yield' : 'mean'
    }).reset_index()

In [132]:
df_import_export_crops

,area_crops,item_crops,year,area_code,item_code_crops,area_harvested,production,yield
0,Afghanistan,"Almonds, with shell",1975,2,221,0.000000,0.000000,0.000000
1,Afghanistan,"Almonds, with shell",1976,2,221,1966.666667,3266.666667,5536.666667
2,Afghanistan,"Almonds, with shell",1977,2,221,2000.000000,3000.000000,5000.000000
3,Afghanistan,"Almonds, with shell",1978,2,221,2000.000000,4000.000000,6666.666667
4,Afghanistan,"Almonds, with shell",1979,2,221,2000.000000,3500.000000,5833.333333
...,...,...,...,...,...,...,...,...
571860,Zimbabwe,Wheat,2013,181,15,7251.000000,16988.666667,7810.000000
571861,Zimbabwe,Wheat,2014,181,15,7377.000000,16198.333333,7319.333333
571862,Zimbabwe,Wheat,2015,181,15,7371.333333,15100.666667,6828.666667
571863,Zimbabwe,Wheat,2016,181,15,7364.666667,14002.666667,6338.000000


In [ ]:
df_cereals = df_import_export_trades[df_import_export_trades['parent_group']==10]

### Group by countries

In [7]:
# Load the dataset
df_country = pd.read_csv('Data/GroupsCountry.csv')
df_country.columns = map(lambda name : name.lower().replace(' ', '_'), df_country.columns) #Column names to lower case
df_country.head()

,country_group_code,country_group,country_code,country,m49_code,iso2_code,iso3_code
0,5100,Africa,4,Algeria,12,DZ,DZA
1,5100,Africa,7,Angola,24,AO,AGO
2,5100,Africa,53,Benin,204,BJ,BEN
3,5100,Africa,20,Botswana,72,BW,BWA
4,5100,Africa,233,Burkina Faso,854,BF,BFA


In [10]:
# Create a dictionnary that regroup all the countries in one country group
dicts = {}
list_group = df_country['country_group_code'].unique()
keys = list_group
for i in range(len(list_group)):
    dicts[list_group[i]]= df_country[df_country['country_group_code']==list_group[i]]['country_code'].unique()

In [11]:
# Create dictionnary for name + subregions
dicts_country = {}
keys = list_group
for i in range(len(list_group)):
    dicts_country[list_group[i]] = df_country['country_group'].unique()[i]

In [12]:
# Display all the suregions and their name
pd.DataFrame.from_dict(dicts_country,orient='index',columns=['regions']).head(15)

,regions
5100,Africa
5200,Americas
5300,Asia
5501,Australia and New Zealand
5206,Caribbean
5204,Central America
5301,Central Asia
5101,Eastern Africa
5302,Eastern Asia
5401,Eastern Europe


### Group by crops

In [13]:
#Quickly load the data : 
group_crops_df = pd.read_csv('data/GroupCrops.csv')
group_crops_df.columns = map(lambda name : name.lower().replace(' ', '_'), group_crops_df.columns) #Column names to lower case
group_crops_df.shape

(2415, 9)

In [14]:
# Load the UN Comtrade Commodity Classifications :
UNCCC_df = pd.read_excel('data/UNCCC.xlsx')

In [15]:
HS12_df = UNCCC_df[UNCCC_df.Classification=='H4']

In [16]:
# df group crops with the nan values removed
group_crops_cleaned_df = group_crops_df[~group_crops_df['hs12_code'].isnull()]

In [17]:
# If the items is assigned mutliple labels, we only keep the first one
HS12_Code = group_crops_cleaned_df['hs12_code'].str.split(', ',expand=True).loc[:,0]

In [18]:
group_crops_cleaned_df = pd\
                            .concat([group_crops_cleaned_df,HS12_Code],axis=1)\
                            .drop(['factor','hs_code','hs07_code','cpc_code','hs12_code'],axis=1)

In [19]:
group_crops_cleaned_df.rename(columns={0:'hs12_code'},inplace=True)

In [20]:
group_crops_cleaned_df['parent_group'] = group_crops_cleaned_df['hs12_code'].str[:2].astype(int)
group_crops_cleaned_df['child_group'] = group_crops_cleaned_df['hs12_code'].str[:4].astype(int)

In [21]:
# Final dataset used for the mapping of each crop : 
group_crops_cleaned_df = group_crops_cleaned_df\
                            .drop(['item_group_code','item_group'],axis=1)\
                            .groupby(['item','item_code','hs12_code','parent_group','child_group'])\
                            .count()\
                            .reset_index()

In [22]:
group_crops_cleaned_df

,item,item_code,hs12_code,parent_group,child_group
0,Agave fibres nes,800,530500,53,5305
1,Alfalfa meal and pellets,862,121410,12,1214
2,Almonds shelled,231,080212,8,802
3,"Almonds, with shell",221,080211,8,802
4,Animals live nes,1171,010611,1,106
...,...,...,...,...,...
558,"Wool, shoddy",1007,510400,51,5104
559,Yams,137,071430,7,714
560,Yautia (cocoyam),135,071450,7,714
561,Yoghurt,891,040310,4,403


In [ ]:
# Merging on item_code only for the trade_df at the moment
df_final = trades_df.merge(group_crops_cleaned_df, left_on='item_code',right_on='item_code')
df_final.head()

In [ ]:
def badassfunction(df_import_export,group_country=None,country=None,group_crops=None,crops=None,year=None,by_crop=False,by_country=False):
    """
    From the original dataset, this function will processed the dataset 
    to return a summary of the desired crops, country and year
    INPUTS : 
    - df_import_export
    - group_country
    - country
    - group_crops
    - year
    OUTPUT : 
    - cleaned df
    """
    df_cleaned = df_import_export
    # Extract country
    if group_country != None :
        df_cleaned = df_cleaned[df_cleaned['area_code'].isin(dicts.get(group_country))]
    if country != None:
        df_cleaned = df_cleaned[df_cleaned['country_code']==country]
    if group_crops != None:
        df_cleaned = df_cleaned[df_cleaned['parent_group']==group_crops]
    if crops != None:
        df_cleaned = df_cleaned[df_cleaned['child_group']==crops]
    if year != None: 
        df_cleaned = df_cleaned[df_cleaned['year']==year]
     
    # Add new features
    df_cleaned['diff_quantity'] = df_cleaned['export_quantity'] - df_cleaned['import_quantity']
    df_cleaned['profit'] = df_cleaned['export_value'] - df_cleaned['import_value']

    # If feature is added above then add it name in that list
    list_features = ['export_quantity','export_value','import_quantity','import_value','diff_quantity','profit']
    #Group by item
    if by_crop:
        df_sum = df_cleaned.groupby(['item_x','parent_group','child_group'])[list_features].sum().reset_index()

        df_max = df_cleaned.groupby(['item_x','parent_group','child_group'])[list_features].max().reset_index()   
        df_max_index = df_cleaned.groupby(['item_x','parent_group','child_group'])[list_features].idxmax().reset_index()

        df_min = df_cleaned.groupby(['item_x','parent_group','child_group'])[list_features].min().reset_index()
        df_min_index =  df_cleaned.groupby(['item_x','parent_group','child_group'])[list_features].idxmin().reset_index()
    
        merge_item = 'item_x'
        replace_item = 'area'

    if by_country:
        df_sum = df_cleaned.groupby(['area','area_code'])[list_features].sum().reset_index()

        df_max = df_cleaned.groupby(['area','area_code'])[list_features].max().reset_index()   
        df_max_index = df_cleaned.groupby(['area','area_code'])[list_features].idxmax().reset_index()

        df_min = df_cleaned.groupby(['area','area_code'])[list_features].min().reset_index()
        df_min_index =  df_cleaned.groupby(['area','area_code'])[list_features].idxmin().reset_index()
    
        merge_item = 'area'
        replace_item = 'item_x'
    
    for i in range(len(list_features)):
        df_max_index.loc[:,list_features[i]] = df_import_export.loc[df_max_index.loc[:,list_features[i]],replace_item].values
    for i in range(len(list_features)):
        df_min_index.loc[:,list_features[i]] = df_import_export.loc[df_min_index.loc[:,list_features[i]],replace_item].values
    
    # merge max 
    df_max_merged = pd.merge(df_max.loc[:,[merge_item]+list_features],df_max_index.loc[:,[merge_item]+list_features],left_on=merge_item,right_on=merge_item,how='inner',suffixes=('_max','_max_names'))
    # merge min 
    df_min_merged = pd.merge(df_min.loc[:,[merge_item]+list_features],df_min_index.loc[:,[merge_item]+list_features],left_on=merge_item,right_on=merge_item,how='inner',suffixes=('_min','_min_names'))

    # merge total
    df_merged_tot = pd.merge(df_min_merged,df_max_merged, left_on=merge_item,right_on=merge_item)
    
    df_tot = pd.merge(df_sum, df_merged_tot, left_on=merge_item,right_on=merge_item)
    return df_tot